In [1]:
def haversine_1(dist, lat1):
    
    RADIUS = 6371000
    
    # distancia = radio de la tierra * C
    C = dist / RADIUS
    a = (sin(C/2))**2
    lat2 = 2*(asin(a)**2)+lat1
    
    #print(dist)
    #print(C)
    #print(a)
    #print(lat2)
    #print()
    
    return lat2

In [2]:
import os
import cv2
import numpy as np
import glob
import random
import pandas as pd

from math import radians, cos, sin, asin, sqrt, atan

# Load Yolo
net = cv2.dnn.readNet("yolov3_training_last.weights", "yolov3_testing.cfg")

# Name custom object
classes = ["buildings"]

# Images path
#  --> Cambiar para que acceda directamente a las fotos del drive / bucket de GCP de Alfon...
images_path = glob.glob(r"C:\Users\pablo\Documents\OAN Tech\Object Detection\yolo_custom_detection\predictions\*.png")

list_nobj = []

layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 1))

# cm = px * (2.54/96)
px = 96 / 2.54
print(f'Nº de pixels en 1 cm: {px}')
zoom_cm =  2256.994440 / 100

# Latitud : metros --> esto es en el equinocio, creo que no nos vale
REL_LAT = 110574
RADIUS = 6371000   #en metros

# Insert here the path of your images
#random.shuffle(images_path)

#Dataframe donde vamos a guardar los datos de las casas detectadas:
df = pd.DataFrame(columns=['x', 'y', 'center_x', 'center_y', 'lat', 'lon', 'village'])

# loop through all the images
for img_path in images_path: 
    #saco el nombre del fichero
    base = os.path.basename(img_path)
    #tener en cuenta que separa el nombre por "_" para sacar sólo el nombre del pueblo
    name = os.path.splitext(base)[0].split("_")[0]
    
    # Latitud y longitud del origen de la imagen 
        ##### Esto lo tenemos que sacar de la tabla de las imágenes de Alfon.
    lat_0 = 9.946898161535087
    lon_0 = 3.4182956338157897
    
    # Loading image
    img = cv2.imread(img_path)
    
    #Reducimos un poco la escala para ver la imagen entera por pantalla (parámetros fx fy)
    img = cv2.resize(img, None, fx=0.5, fy=0.5)
    height, width, channels = img.shape
    print(name)
    #print(height)
    #print(width)
    #Estas medidas están en pixeles

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    
    for out in outs:
        n_objetos = 0
        
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            if confidence > 0.3:
                # Object detected
                n_objetos += 1
                
                # Centros
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
                
                #Cálculo de las coordenadas de cada una de las casas detectadas
                    # Calculo el nº de cm a los que está el objeto detectado en la imagen respecto del origen
                    # Paso de cm en la imagen a metros en la realidad, en base al zoom 19
                m_x = x / px * zoom_cm    
                m_y = y / px * zoom_cm
                
                #Calculo las coordenadas del punto mediante las aproximaciones:
                #lat_pto = (m_x / REL_LAT) + lat_0  # ESTE CÁLCULO ESTÁ MAL.
                #SI SACO LOS RESULTADOS EN EL MAPA NO COINCIDEN CON LAS CASAS.
                #lon_pto = (0) + lon_0
                
                
                ######################################################
                lat_pto = haversine_1(m_x, lat_0)
                lon_pto = 0
                ######################################################
                
                #Añado los objetos detectados al df
                df2 = pd.DataFrame([[x, y, center_x, center_y, lat_pto, lon_pto, name]],
                                   columns=['x', 'y', 'center_x', 'center_y', 'lat', 'lon', 'village'])
                df= df.append(df2)
                
    list_nobj.append(n_objetos)    
    
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    #font = cv2.FONT_HERSHEY_PLAIN
    
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            #label = str(classes[class_ids[i]])
            label = ""
            color = (255, 0, 0) 
            
            #cv2.rectangle(imagen, start point, end point, color, grosor de la línea)
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
            
            #Si quisieramos escribir el nombre de la clase detectada:
            #cv2.putText(img, label, (x, y + 30), font, 3, color, 2)

    cv2.imshow("Image", img)
    key = cv2.waitKey(0)

print(f'Número de casas por imagen: {list_nobj}')
cv2.destroyAllWindows()

Nº de pixels en 1 cm: 37.79527559055118
Sansi Gando
Número de casas por imagen: [12]


In [3]:
df.head(15)

,x,y,center_x,center_y,lat,lon,village
0,571,486,580,496,9.946898,0,Sansi Gando
0,759,486,768,498,9.946898,0,Sansi Gando
0,760,485,770,498,9.946898,0,Sansi Gando
0,566,513,575,520,9.946898,0,Sansi Gando
0,569,514,579,522,9.946898,0,Sansi Gando
0,249,738,259,747,9.946898,0,Sansi Gando
0,292,737,306,744,9.946898,0,Sansi Gando
0,294,736,308,742,9.946898,0,Sansi Gando
0,276,748,286,759,9.946898,0,Sansi Gando
0,323,755,332,769,9.946898,0,Sansi Gando


In [4]:
df.village.value_counts()

Sansi Gando    12
Name: village, dtype: int64